# Лабораторная работа №3. Лобанов Дмитрий ИУ5-25М
### Цель работы
Изучение продвинутых способов предварительной обработки данных для дальнейшего формирования моделей.
### Задание
1) Выбрать один или несколько наборов данных (датасетов) для решения следующих задач. Каждая задача может быть решена на отдельном датасете, или несколько задач могут быть решены на одном датасете. Просьба не использовать датасет, на котором данная задача решалась в лекции.

2) Для выбранного датасета (датасетов) на основе материалов лекций решить следующие задачи:

i) масштабирование признаков (не менее чем тремя способами);

ii) обработку выбросов для числовых признаков (по одному способу для удаления выбросов и для замены выбросов);

iii) обработку по крайней мере одного нестандартного признака (который не является числовым или категориальным);

iv) отбор признаков:

* один метод из группы методов фильтрации (filter methods);
* один метод из группы методов обертывания (wrapper methods);
* один метод из группы методов вложений (embedded methods).

## Ход работы

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "heart.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "johnsmith88/heart-disease-dataset",
  file_path,
)
df.shape
df.head()

<ipython-input-1-f8b3c8f6d542>:12: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df = kagglehub.load_dataset(


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [ ]:
df_original = df.copy()

Поскольку в датасете все значения - числовые или категориальные, надо добавить новое. Я добавлю строковый признак "заметки врача" - patient_notes

In [ ]:
import random

# Большой список "слов" и "фраз", связанных с медициной
phrases = [
    "chest pain", "dizziness", "shortness of breath", "nausea", "fatigue",
    "heart murmur", "high blood pressure", "low heart rate", "irregular heartbeat",
    "sudden sweating", "pain radiating to left arm", "sharp pain", "mild discomfort",
    "exercise-induced symptoms", "no prior history", "family history of heart disease",
    "occasional palpitations", "elevated cholesterol", "normal ECG", "slight arrhythmia",
    "stress-related chest pressure", "trouble breathing during sleep", "persistent cough",
    "weight gain", "recent viral infection", "anxiety", "tightness in chest",
    "episodes of unconsciousness", "normal blood sugar", "cold extremities",
    "light-headedness", "chest heaviness", "pain while lying down",
    "better with rest", "worse with activity", "recommended cardiology consult"
]

# Функция генерации уникальной текстовой строки
def generate_note():
    selected = random.sample(phrases, k=random.randint(5, 10))
    return ", ".join(selected) + "."

# Применим ко всем строкам датафрейма
df["patient_notes"] = [generate_note() for _ in range(len(df))]

In [ ]:
df["patient_notes"][33]

'cold extremities, recommended cardiology consult, light-headedness, anxiety, normal blood sugar, no prior history, normal ECG, family history of heart disease.'

## Масштабироавние признаков

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

scaler = MinMaxScaler()
df_scaled_minmax = df.copy()
df_scaled_minmax[['age', 'trestbps', 'chol', 'thalach', 'oldpeak']] = scaler.fit_transform(df[['age', 'trestbps', 'chol', 'thalach', 'oldpeak']])
df_scaled_minmax.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target,patient_notes
0,0.479167,1,0,0.292453,0.196347,0,1,0.740458,0,0.161290,2,2,3,0,"nausea, episodes of unconsciousness, irregular..."
1,0.500000,1,0,0.433962,0.175799,1,0,0.641221,1,0.500000,0,0,3,0,"normal ECG, slight arrhythmia, chest heaviness..."
2,0.854167,1,0,0.481132,0.109589,0,1,0.412214,1,0.419355,0,0,3,0,"chest heaviness, persistent cough, light-heade..."
3,0.666667,1,0,0.509434,0.175799,0,1,0.687023,0,0.000000,2,1,3,0,"high blood pressure, fatigue, shortness of bre..."
4,0.687500,0,0,0.415094,0.383562,1,1,0.267176,0,0.306452,1,3,2,0,"stress-related chest pressure, nausea, trouble..."


In [ ]:
scaler = StandardScaler()
df_scaled_standard = df.copy()
df_scaled_standard[['age', 'trestbps', 'chol', 'thalach', 'oldpeak']] = scaler.fit_transform(df[['age', 'trestbps', 'chol', 'thalach', 'oldpeak']])
df_scaled_standard.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target,patient_notes
0,-0.268437,1,0,-0.377636,-0.659332,0,1,0.821321,0,-0.060888,2,2,3,0,"nausea, episodes of unconsciousness, irregular..."
1,-0.158157,1,0,0.479107,-0.833861,1,0,0.255968,1,1.727137,0,0,3,0,"normal ECG, slight arrhythmia, chest heaviness..."
2,1.716595,1,0,0.764688,-1.396233,0,1,-1.048692,1,1.301417,0,0,3,0,"chest heaviness, persistent cough, light-heade..."
3,0.724079,1,0,0.936037,-0.833861,0,1,0.516900,0,-0.912329,2,1,3,0,"high blood pressure, fatigue, shortness of bre..."
4,0.834359,0,0,0.364875,0.930822,1,1,-1.874977,0,0.705408,1,3,2,0,"stress-related chest pressure, nausea, trouble..."


In [ ]:
scaler = RobustScaler()
df_scaled_robust = df.copy()
df_scaled_robust[['age', 'trestbps', 'chol', 'thalach', 'oldpeak']] = scaler.fit_transform(df[['age', 'trestbps', 'chol', 'thalach', 'oldpeak']])
df_scaled_robust.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target,patient_notes
0,-0.307692,1,0,-0.25,-0.437500,0,1,0.470588,0,0.111111,2,2,3,0,"nausea, episodes of unconsciousness, irregular..."
1,-0.230769,1,0,0.50,-0.578125,1,0,0.088235,1,1.277778,0,0,3,0,"normal ECG, slight arrhythmia, chest heaviness..."
2,1.076923,1,0,0.75,-1.031250,0,1,-0.794118,1,1.000000,0,0,3,0,"chest heaviness, persistent cough, light-heade..."
3,0.384615,1,0,0.90,-0.578125,0,1,0.264706,0,-0.444444,2,1,3,0,"high blood pressure, fatigue, shortness of bre..."
4,0.461538,0,0,0.40,0.843750,1,1,-1.352941,0,0.611111,1,3,2,0,"stress-related chest pressure, nausea, trouble..."


## Обработка выбросов

In [ ]:
def remove_outliers(df, columns):
    df_no_outliers = df.copy()
    for col in columns:
        Q1 = df_no_outliers[col].quantile(0.25)
        Q3 = df_no_outliers[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df_no_outliers = df_no_outliers[(df_no_outliers[col] >= lower_bound) & (df_no_outliers[col] <= upper_bound)]
    return df_no_outliers

def replace_outliers_with_median(df, columns):
    df_replace_outliers = df.copy()
    for col in columns:
        Q1 = df_replace_outliers[col].quantile(0.25)
        Q3 = df_replace_outliers[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        median_value = df_replace_outliers[col].median()
        df_replace_outliers[col] = df_replace_outliers[col].apply(
            lambda x: median_value if (x < lower_bound) or (x > upper_bound) else x
        )
    return df_replace_outliers

In [ ]:
numeric_columns = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

df_no_outliers = remove_outliers(df, numeric_columns)
df_replace_outliers = replace_outliers_with_median(df, numeric_columns)

# Статистика до обработки выбросов
print("Статистика до обработки выбросов:")
print(df[numeric_columns].describe())

# Статистика после удаления выбросов
print("\nСтатистика после удаления выбросов:")
print(df_no_outliers[numeric_columns].describe())

# Статистика после замены выбросов на медиану
print("\nСтатистика после замены выбросов на медиану:")
print(df_replace_outliers[numeric_columns].describe())

Статистика до обработки выбросов:
               age     trestbps        chol      thalach      oldpeak
count  1025.000000  1025.000000  1025.00000  1025.000000  1025.000000
mean     54.434146   131.611707   246.00000   149.114146     1.071512
std       9.072290    17.516718    51.59251    23.005724     1.175053
min      29.000000    94.000000   126.00000    71.000000     0.000000
25%      48.000000   120.000000   211.00000   132.000000     0.000000
50%      56.000000   130.000000   240.00000   152.000000     0.800000
75%      61.000000   140.000000   275.00000   166.000000     1.800000
max      77.000000   200.000000   564.00000   202.000000     6.200000

Статистика после удаления выбросов:
              age    trestbps        chol     thalach     oldpeak
count  964.000000  964.000000  964.000000  964.000000  964.000000
mean    54.057054  129.957469  242.134855  149.506224    0.986203
std      9.142455   15.321181   45.094967   22.721506    1.052420
min     29.000000   94.000000  126.

## Обработка текстового некатегориального признака

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

df_text_processing = df.copy()

# Преобразуем текстовые данные (patient_notes) в числовые с помощью TF-IDF
vectorizer = TfidfVectorizer(max_features=10)  # Ограничиваем количество признаков
text_features = vectorizer.fit_transform(df_text_processing['patient_notes'])
df_text_features = pd.DataFrame(text_features.toarray(), columns=vectorizer.get_feature_names_out())

# Добавляем текстовые признаки в DataFrame
df_text_processing = pd.concat([df_text_processing, df_text_features], axis=1)

In [ ]:
df_text_processing.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,...,blood,chest,extremities,heart,history,normal,of,pain,pressure,with
0,52,1,0,125,212,0,1,168,0,1.0,...,0.000000,0.000000,0.0,0.540961,0.00000,0.633717,0.552959,0.000000,0.000000,0.0
1,53,1,0,140,203,1,0,155,1,3.1,...,0.000000,0.488198,0.0,0.566622,0.00000,0.663779,0.000000,0.000000,0.000000,0.0
2,70,1,0,145,174,0,1,125,1,2.6,...,0.000000,0.303335,0.0,0.352062,0.41405,0.000000,0.719741,0.307532,0.000000,0.0
3,61,1,0,148,203,0,1,161,0,0.0,...,0.503207,0.000000,0.0,0.423387,0.00000,0.000000,0.432778,0.369836,0.493408,0.0
4,62,0,0,138,294,1,1,106,0,1.9,...,0.000000,0.594491,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.804102,0.0


In [ ]:
df_text_processing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   age            1025 non-null   int64  
 1   sex            1025 non-null   int64  
 2   cp             1025 non-null   int64  
 3   trestbps       1025 non-null   int64  
 4   chol           1025 non-null   int64  
 5   fbs            1025 non-null   int64  
 6   restecg        1025 non-null   int64  
 7   thalach        1025 non-null   int64  
 8   exang          1025 non-null   int64  
 9   oldpeak        1025 non-null   float64
 10  slope          1025 non-null   int64  
 11  ca             1025 non-null   int64  
 12  thal           1025 non-null   int64  
 13  target         1025 non-null   int64  
 14  patient_notes  1025 non-null   object 
 15  blood          1025 non-null   float64
 16  chest          1025 non-null   float64
 17  extremities    1025 non-null   float64
 18  heart   

## Отбор признаков

In [ ]:
df_filter = df_original.copy()

# Метод фильтрации: корреляция признаков с целевой переменной
corr_matrix = df_filter.corr()
relevant_features = corr_matrix['target'].sort_values(ascending=False)
selected_features = relevant_features[relevant_features > 0.50].index.tolist()
df_filter = df_filter[selected_features]
df_filter.head()

,target
0,0
1,0
2,0
3,0
4,0


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

df_rfe = df_original.copy()

model = LogisticRegression()
numeric_columns = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']  # Признаки для анализа
rfe = RFE(estimator=model, n_features_to_select=2)
fit = rfe.fit(df_rfe[numeric_columns], df_rfe['target'])

selected_features_rfe = df_rfe[numeric_columns].columns[fit.support_]
df_rfe_selected = df_rfe[selected_features_rfe]
df_rfe_selected.head()

,thalach,oldpeak
0,168,1.0
1,155,3.1
2,125,2.6
3,161,0.0
4,106,1.9


In [ ]:
from sklearn.ensemble import RandomForestClassifier

df_important = df_original.copy()

numeric_columns = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

# Обучение модели RandomForest
model = RandomForestClassifier()
model.fit(df_important[numeric_columns], df_important['target'])

feature_importances = model.feature_importances_
numeric_columns_np = np.array(numeric_columns)
important_features = numeric_columns_np[feature_importances > 0.15]

important_features


array(['age', 'chol', 'thalach', 'oldpeak'], dtype='<U8')